In [7]:
import pandas as pd
from pymongo import MongoClient

In [58]:
# Connect to mongodb and create a new database
client = MongoClient("mongodb://localhost:27017/")
database = client["MedGraph"]

# Base pubmed url: https://pubmed.ncbi.nlm.nih.gov/ + paper_id

In [18]:
def _connect_mongo():
    return MongoClient("mongodb://localhost:27017/")

def read_mongo(db, collection):
    client = _connect_mongo()
    database = client[db]
    col = database[collection]

    df =  pd.DataFrame(list(col.find({})))
    return df

if __name__ == '__main__':
    df = read_mongo('MedGraph', 'Dataset2000Entries')
    df.to_csv('../data/dataset2000entries.csv', index=False)

In [64]:
dataframe = pd.read_csv('../data/dataset2000entries.csv')

# Upload Authors list as mongodb collection
dataframe["authors"] = dataframe["authors"].apply(lambda x: x.split(", "))
collection = database["Authors"]
authors_set = set()
for authors in dataframe["authors"]:
    for author in authors:
        authors_set.add(author)
for author in authors_set:
    collection.insert_one({"author": author})


# Upload MeshTerms list as mongodb collection
dataframe["mesh_terms"] = dataframe["mesh_terms"].apply(lambda x: x.split("; "))
collection = database["MeshTerms"]
for terms in dataframe["mesh_terms"]:
    for term in terms:
        _id = term.split(":")[0]
        try:
            collection.insert_one({"_id": _id, "term_name" : term.split(":")[1]})
        except: continue


# Upload Areas list as mongodb collection
dataframe["Areas"] = dataframe["Areas"].apply(lambda x: x.split("; "))
collection = database["Areas"]
areas_set = set()
for areas in dataframe["Areas"]:
    for area in areas:
        areas_set.add(area)
        
for area in areas_set:
    collection.insert_one({"area": area})


# Upload Categories list as mongodb collection
dataframe["Categories"] = dataframe["Categories"].apply(lambda x: x.split("; "))
collection = database["Categories"]
categories_set = set()
for categories in dataframe["Categories"]:
    for category in categories:        
        categories_set.add(category[:-5])
        
for category in categories_set:
    collection.insert_one({"category": category})


# dataframe["keywords"] = dataframe["keywords"].apply(lambda x: x.split("; "))

In [86]:
# link authors id to paper author list
dataset2000_collection = database["Dataset2000Entries"]

authors_collection = database["Authors"]
for document in dataset2000_collection.find({}):
    document_id = document["_id"]
    authors = document["authors"].split(", ")
    authors_list = []
    for item in authors:
        cursor = authors_collection.find({"author" : item})
        key = list(cursor)[0]["_id"]
        authors_list.append(key)
    dataset2000_collection.update_one({"_id": document_id}, {"$set": { "authors": authors_list}})

In [87]:
# link mesh terms id to paper mesh terms list
dataset2000_collection = database["Dataset2000Entries"]

for document in dataset2000_collection.find({}):
    document_id = document["_id"]
    mesh_terms = document["mesh_terms"].split("; ")
    terms_list = []
    for term in mesh_terms:
        term_key = term.split(":")[0]
        terms_list.append(term_key)
    dataset2000_collection.update_one({"_id": document_id}, {"$set": { "mesh_terms": terms_list}})

In [88]:
# link areas id to paper areas list
dataset2000_collection = database["Dataset2000Entries"]

areas_collection = database["Areas"]
for document in dataset2000_collection.find({}):
    document_id = document["_id"]
    areas = document["Areas"].split("; ")
    areas_list = []
    for area in areas:
        cursor = areas_collection.find({"area" : area})
        key = list(cursor)[0]["_id"]
        areas_list.append(key)
    dataset2000_collection.update_one({"_id": document_id}, {"$set": { "Areas": areas_list}})

In [90]:
# link categories id to paper categories list
dataset2000_collection = database["Dataset2000Entries"]

categories_collection = database["Categories"]
for document in dataset2000_collection.find({}):
    document_id = document["_id"]
    categories = document["Categories"].split("; ")
    categories_list = []
    for category in categories:    
        cursor = categories_collection.find({"category" : category[:-5]})
        key = list(cursor)[0]["_id"]
        categories_list.append(key)
    dataset2000_collection.update_one({"_id": document_id}, {"$set": { "Categories": categories_list}})